In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from pathlib import Path
from tqdm.notebook import tqdm

#
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import matplotlib.font_manager
import plotly.io as pio
pio.renderers.default = 'iframe'

#
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from pycaret.classification import ClassificationExperiment, models

#
from emidec_lib.dataset import extract_radiomics_dataset
from emidec_lib.utils import (
    calculate_radiomics_features, 
    get_feature_classes, 
    get_clinical_features,
    load_image,
    load_mask,
    filter_lvm
)
from emidec_lib.visualization import mask_overlay, plot_4_slices, plot_slice_x

ModuleNotFoundError: No module named 'emidec_lib'

**Q**: How many machine learning specialists does it take to change a light bulb?

**A**: Just one, but they require a million light bulbs to train properly.

**Q**: How many machine learning specialists does it take to change a fluorescent light bulb?

**A**: That wasn't in the training data!

# Load EMIDEC dataset

The Emidec dataset is a new publicly available DE-MRI database with associated clinical information consisting of 150 exams. The data of each clinical exam are divided into two parts:

- MR images composed of a series of DE-MRI in short axis orientation that cover the left ventricle of the heart, with the corresponding ground truths;
- A text file with the clinical information.

There is an unbalanced distribution between normal (1/3) and pathological (2/3) cases, corresponding roughly to real life in an MRI department. The targeted cohort is any patient admitted in a cardiac emergency department with symptoms of a heart attack. Each group was clearly defined according to physiological parameters and the presence or absence of a disease area on DE-MRI. All pathological cases are patients with acute MI, and the MRI exam was done within one month of the angioplasty procedure. Patients with multiple pathologies were discarded.


## Radiomics

![alt text](resources/radiomics.jpg)

Radiomics is an emerging field in medical imaging that involves the extraction of a large number of features from medical images using data-characterization algorithms. These features, which can be shape, intensity, texture, or wavelet features, were originally designed to quantify the tumor phenotype. Nowadays, this approach is particularly useful in medical image-based phenotyping.

https://pyradiomics.readthedocs.io/en/latest/

In [ ]:
dataset = 'training'
data_path = Path('/data/Lab8_Classification')
data_df = extract_radiomics_dataset(data_path, dataset=dataset, map_categorical=True)
feature_classes = get_feature_classes(data_df.columns)

In [ ]:
data_df

# Train/test split

The train/test split is essential in developing machine learning (ML) models for healthcare for several key reasons:

1. **Generalization**: It evaluates the model's performance on unseen data, ensuring it can generalize beyond the specific data it was trained on.
2. **Overfitting Prevention**: Helps detect overfitting by comparing the model's performance on training data versus unseen test data. Overfitting occurs when a model learns from noise or random fluctuations in the training data, reducing its effectiveness on new data.
3. **Model Selection and Tuning**: Facilitates the comparison of different models and settings on a consistent dataset, aiding in the selection of the most effective model.
4. **Robustness Validation**: Ensures the model's reliability across different patient demographics, disease states, and clinical environments, a critical factor in medical applications.
5. **Regulatory and Ethical Compliance**: Demonstrating effective performance on a separate test set is often required for regulatory approval. It also supports ethical considerations by ensuring models do not amplify biases.
6. **Supports Incremental Learning**: Allows for the structured evaluation of model updates, ensuring improvements are validated before deployment in a clinical setting.

In summary, the train/test split is vital for creating reliable, effective, and ethically responsible ML models in healthcare, ensuring they perform well in real-world conditions and comply with regulatory standards.

### Cross-validation

<img src="resources/cross-validation.png" align="left" width="500" />

In [ ]:
seed = 42
target = 'Pathological'

In [ ]:
train_df, test_df = train_test_split(data_df, test_size=0.2, random_state=seed, stratify=data_df[target])

# Data visualization

In [ ]:
plt.figure()

sns.violinplot(train_df, x='Pathological', y='clinical_Age', hue='clinical_Sex')

## Task 1: 

Play around with plotting different variables.

##### Q1: What information do you get about the dataset?

The dataset has 150 samples with unbalanced classes (1/3 normal, 2/3 pathological). It contains clinical features like age and sex plus radiomics features extracted from MRI images. This is a binary classification problem for detecting some heart condition.


##### Q2: Can you identify properties that are predictive for myocardial infaction?

In [ ]:
columns = ['clinical_Troponin', 'clinical_NTProBNP', 'clinical_Tobacco', 'clinical_Overweight', target]
fig = px.parallel_coordinates(train_df[columns], color=target)
fig.show()

## Evaluation metrics

Accuracy = $\frac{TP + TN}{TP + TN + FP + FN}$

Recall = $\frac{TP}{TP + FN}$

Precision = $\frac{TP}{TP + FP}$

F1 = $\frac{2 * Precision * Recall}{Precision + Recall}$

## Machine learning models

1. Logistic Regression (https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression)
2. Ada Boost Classifier (https://scikit-learn.org/stable/modules/ensemble.html#adaboost)
3. Random Forest Classifier (https://scikit-learn.org/stable/modules/ensemble.html#random-forests-and-other-randomized-tree-ensembles)
4. Support Vector Machine (https://scikit-learn.org/stable/modules/svm.html#classification)
5. Ridge Classifier (https://scikit-learn.org/stable/modules/linear_model.html#ridge-regression-and-classification)

https://pycaret.readthedocs.io/en/latest/index.html

## Task 2

##### Q3: Select two of the classifiers. Briefly describe the concept and what special property this classfier has.

The Logistic Regression Classifier, is a linear classifier that uses a sigmoid function to output probabilities. The coefficients are interpretable which is useful for understand feature importance. 

Random forest is an ensemble method that combines multiple decision trees. It can handle non-linear relationships and automatically provides feature importance scores. 

# Model training (Clinical features)

Along with MRI, clinical characteristics are provided in a text file. These characteristics are: sex, age, tobacco (yes, no, or former smoker), overweight (body mass index (BMI) higher than 25), arterial hypertension (Y/N), diabetes (Y/N), familial history of coronary artery disease (Y/N), electrocardiogram (ECG) (ST+ (STEMI) or not), troponin (value), Killip max (between 1 and 4), ejection fraction of the left ventricle from echocardiography (value), and NTproBNP (value).

https://www.mdpi.com/2306-5729/5/4/89

In [ ]:
clinical_columns = feature_classes['clinical']

In [ ]:
clinical_experiment = ClassificationExperiment()
clinical_experiment.setup(
    data=train_df[clinical_columns + [target]],
    test_data=test_df[clinical_columns + [target]],
    target=target,
    normalize=True,
    normalize_method="zscore",
    fix_imbalance=True,
    remove_outliers=False,
    session_id=seed,
    fold=10
)

In [ ]:
models = ['lr', 'ada', 'rf', 'svm', 'ridge']
clinical_best = clinical_experiment.compare_models(include=models, sort='F1')

In [ ]:
clinical_model = clinical_experiment.create_model("lr")

In [ ]:
clinical_experiment.evaluate_model(clinical_model)

## Task 3
Look at the dicision boundaries of different classifiers. (Feature One, Feature 2 are the two main axes of the PCA)

##### Q4: Do you find a reason why lr is the best?

With only 150 samples, simpler models usually generalize better than complex ones. The features are probably linearly separable after normalization. LR avoids overfitting that more complex models might suffer from with this small dataset size.


# Model training

In [ ]:
lvm_columns = list(filter(lambda c: 'LVM_' in c, train_df.columns))

In [ ]:
radiomics_experiment = ClassificationExperiment()
radiomics_experiment.setup(
    data=train_df[clinical_columns + lvm_columns + [target]],
    test_data=test_df[clinical_columns + lvm_columns + [target]],
    target=target,
    normalize=True,
    normalize_method="zscore",
    fix_imbalance=True,
    remove_outliers=False,
    session_id=seed,
    fold=10
)

In [ ]:
models = ['lr', 'ada', 'rf', 'svm', 'ridge']
radiomics_best = radiomics_experiment.compare_models(include=models, sort='F1')

In [ ]:
radiomics_model = radiomics_experiment.create_model("lr")

In [ ]:
radiomics_experiment.evaluate_model(radiomics_model)

# Task 4: 
Look what changed by using the image based features (radiomics). First order skewness (look below) is the most relevant feature. 
##### Q5: Why could this be the case? Does it make sense from what we know about the MRI data? (Also look at the example plots below to find a reasoning.)


## Interpretation

### Skewness (First-order)

https://pyradiomics.readthedocs.io/en/latest/features.html#radiomics.firstorder.RadiomicsFirstOrder

![title](resources/skewness.png)

$$\textit{skewness} = \displaystyle\frac{\mu_3}{\sigma^3} =
\frac{\frac{1}{N_p}\sum^{N_p}_{i=1}{(\textbf{X}(i)-\bar{X})^3}}
{\left(\sqrt{\frac{1}{N_p}\sum^{N_p}_{i=1}{(\textbf{X}(i)-\bar{X})^2}}\right)^3}
$$

In [ ]:
plt.figure()

sns.violinplot(train_df, x='clinical_Sex', y='LVM_firstorder_Skewness', hue='Pathological')

### Plot a normal case

In [ ]:
case_normal = 'Case_N070'
image_normal, _ = load_image(data_path / dataset, case_normal)
mask_normal = load_mask(data_path / dataset, case_normal)
image_normal, mask_normal = image_normal.T, mask_normal.T
plot_4_slices(image_normal[:, 100:150, 45:100])

### Plot a pathological case

In [ ]:
case_patho = 'Case_P066'
image_patho, _ = load_image(data_path / dataset, case_patho)
mask_patho = load_mask(data_path / dataset, case_patho)
image_patho, mask_patho = image_patho.T, mask_patho.T
plot_4_slices(image_patho[:, 100:150, 100:200])

### Plot intensity distributions

In [ ]:
intensities_normal = pd.DataFrame(image_normal[filter_lvm(mask_normal) == 1], columns=['Intensity'])
intensities_normal['Pathological'] = False
intensities_patho = pd.DataFrame(image_patho[filter_lvm(mask_patho) == 1], columns=['Intensity'])
intensities_patho['Pathological'] = True
intensities = pd.concat((intensities_normal, intensities_patho))

In [ ]:
plt.figure()
sns.histplot(intensities, x='Intensity', hue='Pathological')

### Correlation (GLCM)

<img src="resources/glcm.png" align="left" width="500" />

https://pyradiomics.readthedocs.io/en/latest/features.html#module-radiomics.glcm

$$\textit{correlation} = \frac{\sum^{N_g}_{i=1}\sum^{N_g}_{j=1}{p(i,j)ij-\mu_x\mu_y}}{\sigma_x(i)\sigma_y(j)}$$

In [ ]:
plt.figure()

sns.violinplot(train_df, x='clinical_Sex', y='LVM_glcm_Correlation', hue='Pathological')

## Task 5: 

##### Q6: What does the GLCM_correlation compute, what texture is an example for a high/low GLCM_correlation?

GLCM correlation measures how predictably neighboring pixels relate to each other. High correlation means smooth, uniform textures while low correlation indicates irregular, heterogeneous patterns. Normal tissue would be more uniform than damaged tissue.

## Final Questions

##### Q7: What are radiomics?
Radiomics extracts quantitative features from medical images using algorithms. It converts images into numerical data for machine learning analysis

##### Q8: Why is train/test split so important?
It prevents overfitting by evaluating on unseen data. This gives realistic performance estimates for real-world deployment.

##### Q9: What is cross-validation?
Cross-validation splits training data into folds, trains on some and validates on others repeatedly. This provides more robust performance estimates than single splits.

##### Q10: How can radiologists use the insights we gained with the datanalysis performed above?
They could use these quantitative features as additional diagnostic markers. The automated analysis could help standardize interpretations and catch subtle patterns humans might miss.